In [ ]:
# | include: false
# | default_exp init

# init

> Initialise a new Sciflow Project. Ensures the right files and environment variables are created for using `sciflow`.

In [ ]:
%load_ext autoreload
%autoreload 2

# Imports

In [ ]:
# | export

import os
from pathlib import Path

from fastcore.script import Param, call_parse

from sciflow.utils import prepare_env

# Test Setup

In [ ]:
examples_root = Path(Path(".").resolve(), "examples")
project_root = Path(".").resolve().parent
sciflow_dir = Path("test/.sciflow")
test_root = Path("test")
env_path = Path(sciflow_dir, "env")

In [ ]:
# | export

env_file_blank = """export USER=
export SCIFLOW_BUCKET=
"""

# `write_env_file`

In [ ]:
# | export


def write_env_file(sciflow_dir: Path = None):
    if sciflow_dir is None:
        sciflow_dir = Path("~/.sciflow").expanduser()
    env_path = Path(sciflow_dir, "env").resolve()
    if not env_path.exists():
        if not sciflow_dir.exists():
            os.mkdir(sciflow_dir)
        with open(env_path, "w") as env_file:
            env_file.write(env_file_blank)
        print(f"Wrote new SciFlow environment file to: {env_path}")
    else:
        print(f"Skipping SciFlow environment file creation - already exists")

In [ ]:
if env_path.exists():
    env_path.unlink()

write_env_file(sciflow_dir)

with open(env_path, "r") as env_file:
    env_file_contents = env_file.read()
assert env_file_contents == env_file_blank

write_env_file()

with open(env_path, "r") as env_file:
    env_file_contents = env_file.read()
assert env_file_contents == env_file_blank

with open(env_path, "w") as env_file:
    env_file.write("test")
write_env_file(sciflow_dir)
with open(env_path, "r") as env_file:
    env_file_contents = env_file.read()
assert "test" == env_file_contents

env_path.unlink()
assert not env_path.exists()

write_env_file(sciflow_dir)

Wrote new SciFlow environment file to: /home/sagemaker-user/git/sciflow/nbs/test/.sciflow/env
Skipping SciFlow environment file creation - already exists
Skipping SciFlow environment file creation - already exists
Wrote new SciFlow environment file to: /home/sagemaker-user/git/sciflow/nbs/test/.sciflow/env


# `read_env_file`

In [ ]:
# | export


def read_env_file(sciflow_dir: Path = None):
    if sciflow_dir is None:
        sciflow_dir = Path("~/.sciflow").expanduser()
    env_path = Path(sciflow_dir, "env").resolve()
    try:
        with open(env_path, "r") as env_file:
            lines = env_file.readlines()
    except FileNotFoundError:
        raise FileNotFoundError(
            "The Sciflow environment file is missing - have you run sciflow_init?"
        )
    return lines

In [ ]:
empty_env_lines = ["export USER=\n", "export SCIFLOW_BUCKET=\n"]

# Unused??
env_lines_wout_ppath = [
    "export USER='sciflow'\n",
    "export SCIFLOW_BUCKET=somebucket\n",
]

env_lines_w_1_ppath = [
    "export USER='sciflow'\n",
    f"export PYTHONPATH=$PYTHONPATH:{str(project_root.resolve())}\n",
    "export SCIFLOW_BUCKET=somebucket\n",
]

env_lines_w_2_ppath = [
    "export USER='sciflow'\n",
    f"export PYTHONPATH=$PYTHONPATH:{str(project_root.resolve())}:{str(examples_root.resolve())}\n",
    "export SCIFLOW_BUCKET=somebucket\n",
]

In [ ]:
assert empty_env_lines == read_env_file(sciflow_dir)

# `edit_pythonpath`

In [ ]:
# | export


def edit_pythonpath(env_lines, dir_to_add: Path):
    dir_str = str(dir_to_add.resolve())
    existing_ppath = [p for p in env_lines if p.find("PYTHONPATH") > -1]

    if len(existing_ppath) == 0:
        new_line = f"export PYTHONPATH=$PYTHONPATH:{dir_str}\n"
        env_lines.append(new_line)
        new_text = "".join(env_lines)
    elif len(existing_ppath) == 1:
        prev_line = existing_ppath[0]
        if prev_line.find(dir_str) == -1:
            new_line = existing_ppath[0].replace(
                "$PYTHONPATH:", f"$PYTHONPATH:{dir_str}:"
            )
            new_text = "".join(env_lines).replace(prev_line, new_line)
        else:
            new_text = "".join(env_lines)
    else:
        raise ValueError(
            "Env file is malformed - only 1 PYTHONPATH entry should be present"
        )
    return new_text

In [ ]:
env_vars = sorted(
    [
        "USER",
        "SCIFLOW_BUCKET",
        "PYTHONPATH",
    ]
)

In [ ]:
env_lines = read_env_file(sciflow_dir)
env_lines

['export USER=\n', 'export SCIFLOW_BUCKET=\n']

In [ ]:
env_lines = read_env_file(sciflow_dir)
assert empty_env_lines == env_lines
observed_vars = [
    x.split("=")[0].split(" ")[1]
    for x in edit_pythonpath(env_lines, examples_root).split("\n")
    if x.find("export") > -1
]
assert env_vars == sorted(observed_vars)

# `write_edited_pythonpath`

In [ ]:
# | export


def write_edited_pythonpath(project_root: Path, sciflow_dir: Path = None):
    if sciflow_dir is None:
        sciflow_dir = Path("~/.sciflow").expanduser()
    env_path = Path(sciflow_dir, "env").resolve()
    env_lines = read_env_file(sciflow_dir)
    new_text = edit_pythonpath(env_lines, project_root)
    with open(env_path, "w") as env_file:
        env_file.write(new_text)

In [ ]:
env_lines_w_ppath = [
    "export USER=\n",
    "export SCIFLOW_BUCKET=\n",
    f"export PYTHONPATH=$PYTHONPATH:{str(test_root.resolve())}\n",
]

In [ ]:
write_edited_pythonpath(test_root, sciflow_dir)

In [ ]:
assert read_env_file(sciflow_dir) == env_lines_w_ppath

# CLI Commands

## `sciflow_init`

In [ ]:
# | export


@call_parse
def sciflow_init(
    project_root: Param("The root directory of the project", Path) = None,
    sciflow_dir: Param("The sciflow env directory", Path) = None,
):
    if project_root is None:
        project_root = Path(".").resolve()
    # TODO - Get latest templates files from web - if has internet connection

    # Create sciflow env file if it doesn't exist
    write_env_file(sciflow_dir)

    # Add project root to PYTHONPATH environment variable
    write_edited_pythonpath(project_root, sciflow_dir)

    prepare_env()

In [ ]:
sciflow_init(examples_root, sciflow_dir)

Skipping SciFlow environment file creation - already exists


In [ ]:
with open(env_path, "r") as env_file:
    env_file_contents = env_file.readlines()

In [ ]:
assert (
    f"export PYTHONPATH=$PYTHONPATH:{str(examples_root.resolve())}:{str(test_root.resolve())}"
    == env_file_contents[-1].strip()
)

In [ ]:
with open(env_path, "w") as env_file:
    env_file.write("".join(env_file_contents[:-1]))